In [1]:
# Pythonで訓練をする
# deviceはcuda or cpu or mps
#  mps: apple silicon
# out of memoryが発生した際には、batch_sizeを減らす


# 途中から計算するときには，modelディレクトリに checkpoint_xxx.pt(xxxは計算済みのepoch数)とcheckpoint_best.pt が存在すること


model.py

- emb_size    
  単語のエンベッドサイズ
- nhead
  transformerのMultiHeadAttentionのヘッド数
- ffn_hid_dim    
  

In [2]:
!python ./model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 1024 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir model \
  --tensorboard_logdir logs \
  --prefix translation \
  --source_lang eng \
  --target_lang fra \
  --train_file ../dataset/train.jsonl \
  --valid_file ../dataset/val.jsonl

num_epochs:100
Epoch: 1, Train loss: 2.971, Val loss: 2.371, Epoch time = 790.949s
Epoch: 2, Train loss: 2.369, Val loss: 2.166, Epoch time = 788.740s
Epoch: 3, Train loss: 2.227, Val loss: 2.084, Epoch time = 789.483s
Epoch: 4, Train loss: 2.144, Val loss: 2.010, Epoch time = 788.312s
Epoch: 5, Train loss: 2.081, Val loss: 1.942, Epoch time = 788.076s
Epoch: 6, Train loss: 2.026, Val loss: 1.883, Epoch time = 789.471s
Epoch: 7, Train loss: 1.978, Val loss: 1.832, Epoch time = 788.610s
Epoch: 8, Train loss: 1.935, Val loss: 1.792, Epoch time = 788.455s
Epoch: 9, Train loss: 1.896, Val loss: 1.742, Epoch time = 789.635s
Epoch: 10, Train loss: 1.861, Val loss: 1.704, Epoch time = 788.345s
Epoch: 11, Train loss: 1.828, Val loss: 1.671, Epoch time = 789.236s
Epoch: 12, Train loss: 1.797, Val loss: 1.637, Epoch time = 788.691s
Epoch: 13, Train loss: 1.768, Val loss: 1.603, Epoch time = 788.918s
Epoch: 14, Train loss: 1.741, Val loss: 1.573, Epoch time = 787.437s
Epoch: 15, Train loss: 1.715

In [ ]:
# javaのDJLで使えるようにモデルファイルを変換する

In [1]:
!python convert.py  \
    --model_file=model/checkpoint_best.pt \
    --model_script=model/script.pt \
    --encoder=model/encoder.pt \
    --decoder=model/decoder.pt \
    --positional_encoding=model/positional_encoding.pt \
    --generator=model/generator.pt \
    --src_tok_emb=model/src_tok_emb.pt \
    --tgt_tok_emb=model/tgt_tok_emb.pt \
    --vocab_src=model/vocab_src.txt \
    --vocab_tgt=model/vocab_tgt.txt \
    --params=model/params.json \
    --device=cpu
